# NumPy + Numba

In [1]:
from numba import (
    __version__ as numba_version,
    njit,
    TypingError,
)
from numpy import (
    __version__ as numpy_version,
    absolute,
    ascontiguousarray,
    asfortranarray,
    argmin,
    array,
    empty,
    inf,
    zeros,
)
from numpy.linalg import multi_dot
from numpy.random import random
from time import time

print("NumPy version of this slideshow is", numpy_version)
print("numba version of this slideshow is", numba_version)

NumPy version of this slideshow is 1.20.3
numba version of this slideshow is 0.54.1


Should I use `numpy` if I have C and Fortran? Nothing can be faster than these guys.

Because `numpy` uses [AVX512](https://github.com/numpy/numpy/blob/85df388d344f4ebd70921dca0bc723770e05a37b/numpy/core/src/common/simd/avx512/arithmetic.h#L160..L171) when it is supported by your computer.

There is even a file called [simd.inc.src](https://github.com/numpy/numpy/blob/main/numpy/core/src/umath/simd.inc.src#L756) with different SIMD tricks.

You should spend a lot of time to outperform [numpy.dot](https://github.com/numpy/numpy/blob/de06954b19a6c0980d48e4397adabfd15d808660/numpy/core/src/common/cblasfuncs.c#L208..L700) by hand because it uses [BLAS](https://www.netlib.org/blas/).

Should I use `numba` if I already have a highly optimized `numpy`?

First, make sure that you use `numpy` properly.
Consider the following example.

In [2]:
a = random((1_000, 1_000))
b = random((1_000, 20_000))
c = random((20_000, 1_000))
start = time()
result = (a @ b) @ c
print(f"Matrices multiplication took {time() - start:>.2}s")
start = time()
result = multi_dot((a, b, c))
print(f"`numpy.multi_dot` took       {time() - start:>.2}s")

del result

Matrices multiplication took 1.0s
`numpy.multi_dot` took       0.46s


Can it be even faster?

In theory, yes.
At least, you can avoid consumation of additional memory for repeated operations.

In [3]:
start = time()
result = empty((a.shape[0], c.shape[1]), a.dtype)
start = time()
result = multi_dot((a, b, c), out=result)
print(f"`numpy.multi_dot` with `out` took {time() - start:>.2}s")

del result

`numpy.multi_dot` with `out` took 0.49s


## Can `numba` help us here?

In [4]:
@njit
def jit_multi_dot(x, y, z):
    return multi_dot((a, b, c))

try:
    jit_multi_dot(a, b, c)
except TypingError as e:
    print(e)

del a
del b
del c

Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'multi_dot': Cannot determine Numba type of <class 'function'>

File "../../../../tmp/ipykernel_1099/1606011171.py", line 3:
<source missing, REPL/exec in use?>



Not all `numpy` abilities are supported.
See [Supported NumPy features](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html) for more details.

## Which features are also not supported?

One of the most painful losses here is an `axis` parameter

In [5]:
@njit
def jit_min(x):
    return x.min(axis=0)

try:
    print(jit_min(array([ [1, 2], [3, 4] ])))
except TypingError as e:
    print(f"{str(e)[:350]}...")

Failed in nopython mode pipeline (step: nopython frontend)
- Resolution failure for literal arguments:
AssertionError()
- Resolution failure for non-literal arguments:
AssertionError()

During: resolving callee type: BoundFunction(array.min for array(int64, 2d, C))
During: typing o...


This is my favorite feature.
It gives half of my code performance.

Again. Why should I use `numba`?

Consider a problem of finding an index of a minimal element in big array

In [6]:
@njit(nogil=True)
def jit_argmin(x):
    index, result = 0, x[0]
    for i in range(1, x.size):
        if x[i] < result:
            index, result = i, x[i]
    return index

a = random(100_000_000)

start = time()
index = argmin(a)
print(f"`numpy.argmin` took             {time() - start:>.2}s")

start = time()
index = jit_argmin(a)
print(f"JIT-powered `argmin` took       {time() - start:>.2}s")

del a

`numpy.argmin` took             0.063s
JIT-powered `argmin` took       0.24s


Imagine a situation when you have a big array and do not want to (or simply can not) consume more RAM nor modify it.

In [7]:
def preprocessor(x):
    return x ** 3 + absolute(x) ** -1.5

jit_preprocessor = njit(nogil=True)(preprocessor)

@njit(nogil=True)
def jit_argmin(x):
    index, result = 0, jit_preprocessor(x[0])
    for i in range(1, x.size):
        element = jit_preprocessor(x[i])
        if element < result:
            index, result = i, element
    return index

a = random(100_000_000)

start = time()
index = argmin(preprocessor(a))
print(f"`numpy.argmin` took                  {time() - start:>.2}s")

start = time()
index = argmin(jit_preprocessor(a))
print(f"JIT preprocessor allows to make it   {time() - start:>.2}s")

start = time()
index = jit_argmin(a)
print(f"JIT `argmin` deals with it in        {time() - start:>.2}s")

del a

`numpy.argmin` took                  5.0s
JIT preprocessor allows to make it   3.1s
JIT `argmin` deals with it in        2.7s


That's why `numpy` function have `out` parameter: `numpy.log`, `numpy.absolute`, `numpy.pow`, etc.
Though, you still have to use additional memory for this `out` if you don't want to modify your source array.

# Creating an array

Remember that you cannot use Python types in `nopython=True` mode

In [8]:
@njit(nogil=True)
def f(i):
    temporary_array = zeros((3,), dtype=float)
    return temporary_array[i]

try:
    f(0)
except TypingError as e:
    print(f"{str(e)[:550]}...")

Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function zeros>) found for signature:
 
 >>> zeros(UniTuple(Literal[int](3) x 1), dtype=Function(<class 'float'>))
 
There are 4 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
    With argument(s): '(UniTuple(int64 x 1), dtype=Function(<class 'float'>))':
   Rejected as the implement...


You should use special types from `numba`

In [9]:
from numba import float64

@njit(nogil=True)
def f(i):
    temporary_array = zeros((3,), dtype=float64)
    return temporary_array[i]

print(f(0))

0.0


In [10]:
from numba import f8

@njit(nogil=True)
def f(i):
    temporary_array = zeros((3,), dtype=f8)
    return temporary_array[i]

print(f(0))

0.0


or `numpy`

In [11]:
from numpy import float64

@njit(nogil=True)
def f(i):
    temporary_array = zeros((3,), dtype=float64)
    return temporary_array[i]

print(f(0))

0.0


## Personal recommendation

Usually, you use `numba` for repeated operations called in loops.

Think twice before creating an array.
Maybe you simply need to provide it as an `out` argument.

For example

In [12]:
@njit(nogil=True)
def f(x):
    cache = empty((10, ), dtype=float64)
    for i in range(cache.size):
        cache[i] = x ** i
    return cache

result = empty((100, 10), dtype=float64)
for i in range(result.shape[0]):
    result[i] = f(i)

Should become

In [13]:
@njit(nogil=True)
def f(x, row):
    for i in range(row.size):
        row[i] = x ** i

result = empty((100, 10), dtype=float64)
for i in range(result.shape[0]):
    f(i, result[i])

# Function signature

Specifying types is one of the reason for starters to choose Python instead of another language.

Though, it is a very strong tool to understand your code, fight errors and other beautiful things.

It is not a rarity for us to make a function working only for specific types.
We even have [mypy](http://mypy-lang.org/).

What is so special about types in Numba?

## Basic syntax

You can use a string to specify a signature for your function

In [14]:
@njit("int64(int64)", nogil=True)
def f(x):
    return x

You can use `numba` types for this purpose

In [15]:
from numba import int32, int64

@njit(int32(int64), nogil=True)
def f(x):
    return x

N-dimensional arrays are made by adding `:` to a type because it has an overloaded `__getindex__` method

In [16]:
from numba import float32

@njit(float32[:](float32[:, :]), nogil=True)
def f(x):
    return x[0]

## Data order

Order of data in arrays may affect performance of your application.

There are functions
[numpy.ascontiguousarray](https://numpy.org/doc/stable/reference/generated/numpy.ascontiguousarray.html)
and [numpy.asfortranarray](https://numpy.org/doc/stable/reference/generated/numpy.asfortranarray.html#numpy.asfortranarray)
that can transform the array order.

You can choose an axis which contains continuously stored data by using `::1` instead of `:`.
Obviously, you cannot specify multiple axes with continuous data storage in one array.

In [17]:
from numba import float32

@njit(float32[::1](float32[:, ::1]), nogil=True)
def f(x):
    return x[0]

The first beautiful thing: `numba` will try to prevent you from creating a wrong signature

In [18]:
try:
    @njit(float32[::1](float32[::1, :]), nogil=True)
    def f(x):
        return x[0]
except TypingError as e:
    print(e)

Failed in nopython mode pipeline (step: nopython frontend)
No conversion from array(float32, 1d, A) to array(float32, 1d, C) for '$8return_value.3', defined at None

File "../../../../tmp/ipykernel_1099/1126163961.py", line 4:
<source missing, REPL/exec in use?>

During: typing of assignment at /tmp/ipykernel_1099/1126163961.py (4)

File "../../../../tmp/ipykernel_1099/1126163961.py", line 4:
<source missing, REPL/exec in use?>



Even like this

In [19]:
from numba import float32, complex64

try:
    @njit(float32(float32), nogil=True)
    def f(x):
        if x > 0:
            return complex64(x)
        else:
            return float32(x)
except TypingError as e:
    print(e)

Failed in nopython mode pipeline (step: nopython frontend)
No conversion from complex64 to float32 for '$16return_value.3', defined at None

File "../../../../tmp/ipykernel_1099/489888538.py", line 7:
<source missing, REPL/exec in use?>

During: typing of assignment at /tmp/ipykernel_1099/489888538.py (7)

File "../../../../tmp/ipykernel_1099/489888538.py", line 7:
<source missing, REPL/exec in use?>



What is the difference between those orders?

In [20]:
from numba import float64

@njit(float64(float64[:, ::1]), nogil=True)
def row_major(x):
    result = 0
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result += x[i, j]
    return result

@njit(float64(float64[::1, :]), nogil=True)
def column_major(x):
    result = 0
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result += x[i, j]
    return result

The difference is noticeable

In [21]:
a = ascontiguousarray(random((50_000, 10_000)))
b = asfortranarray(a)

start = time()
row_major(a)
print(f"Row-major took    {time() - start:>.2}s")

start = time()
column_major(b)
print(f"Column-major took {time() - start:>.2}s")

del a
del b

Row-major took    0.55s
Column-major took 1.7s


It can even help `numpy`

In [22]:
a = ascontiguousarray(random((20_000, 20_000)))

start = time()
c = a.min(axis=1)
print(f"Row-major took    {time() - start:>.2}s")

b = asfortranarray(a)
del a
start = time()
c = b.min(axis=1)
print(f"Column-major took {time() - start:>.2}s")

del b
del c

Row-major took    0.18s
Column-major took 0.36s
